In [ ]:
import pypsa
import pandas as pd
import matplotlib.pyplot as plt

Read network

In [ ]:
n=pypsa.Network("../../results/nz_initial/postnetworks/elec_s_10_ec_lc1.0_Co2L_3H_2030_0.071_AB_20export.nc")

In [ ]:
stats = n.statistics()

In [ ]:
n.statistics.optimal_capacity()["Generator"]/1e3

In [ ]:
n.statistics.dispatch()[n.statistics.dispatch()>1e3] / 1e6

In [ ]:
n.statistics.capex()[n.statistics.capex()>1e6] / 1e6